In [5]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import base64
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [6]:
dt = DynamoTable()
try:
    dt.select_table('SkiLifts')
    print(dt)
except:
    dt.create_table(
        table_name='SkiLifts',
        partition_key='Lift',
        partition_key_type='S',
        sort_key="Metadata",
        sort_key_type="S"
)

Table created successfully!


In [7]:
dt.load_json("ski.json")

**TABLE:**
<br>

![SkiResortDataModel.png](images/ski_data01.png)

In [30]:
# Policy 1 - Allow access to all items in the table if the user is authenticated and primary key is Lift 23
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAccessToOnlyItemsMatchingUserID",
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:BatchGetItem",
                "dynamodb:Query",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "dynamodb:BatchWriteItem"
            ],
            "Resource": "arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts",
            "Condition": {
                "ForAllValues:StringEquals": {
                    "dynamodb:LeadingKeys": [
                        "Lift 23"
                    ]
                }
            }
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "dynamodb:ListTables",
            "Resource": "*"
        }
    ]
}
policy_name = 'dynamodb-policy-test-sample'

In [29]:
# Policy N°2 - Allow access to all items in the table if the user is authenticated and primary 
# key is Lift 23 and attributes are Lift, Metadata and Item
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAccessToOnlyItemsMatchingUserID",
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:BatchGetItem",
                "dynamodb:Query",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "dynamodb:BatchWriteItem"
            ],
            "Resource": "arn:aws:dynamodb:us-east-1:089715336747:table/SkiLifts",
            "Condition": {
                "ForAllValues:StringEquals": {
                    "dynamodb:LeadingKeys": [
                        "Lift 23"
                    ],
                "dynamodb:Attributes": [
                    "Lift",
                    "Metadata",
                    "Item"
                    ]
                },
                "StringEqualsIfExists": {
                    "dynamodb:Select": "SPECIFIC_ATTRIBUTES"
                }
            }
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "dynamodb:ListTables",
            "Resource": "*"
        }
    ]
}
policy_name = 'dynamodb-policy-test-sample'

In [31]:
# Update policy or create if it doesn't exist
iam = boto3.client('iam')
try:
    response = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=json.dumps(policy)
    )
    print(f"Policy {policy_name} created successfully. Arn: {response['Policy']['Arn']}")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        #Update policy
        account_id = boto3.client('sts').get_caller_identity().get('Account')
        response = iam.get_policy(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        policy_arn = response['Policy']['Arn']
        # Delete all policy versions except the default version
        response = iam.list_policy_versions(PolicyArn=f"arn:aws:iam::{account_id}:policy/{policy_name}")
        for version in response['Versions']:
            if not version['IsDefaultVersion']:
                response = iam.delete_policy_version(
                    PolicyArn=policy_arn,
                    VersionId=version['VersionId']
                )   
        response = iam.create_policy_version(
            PolicyArn=policy_arn,
            PolicyDocument=json.dumps(policy),
            SetAsDefault=True
        )
        print(f"Policy {policy_name} updated successfully. Arn: {policy_arn}")
    else:
        print("Unexpected error: %s" % e)
    

Policy dynamodb-policy-test-sample updated successfully. Arn: arn:aws:iam::089715336747:policy/dynamodb-policy-test-sample


In [32]:
import boto3
session = boto3.Session(profile_name='089715336747_DynamoAttributes')
dynamo_client = session.client('dynamodb', region_name='us-east-1')

### Policy N° 1

In [ ]:
try:
    dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 20'}, 'Metadata': {'S': 'Static Data'}})
except ClientError as e:
    print("Error:", e.response['Error']['Code'])

In [ ]:
dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}})

### Policy N° 2

In [38]:
try:
    response = dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}})
    print(response['Item'])
except ClientError as e:
    print("Error:", e.response['Error']['Code'])

Error: ExpiredTokenException


In [27]:
dynamo_client.get_item(TableName='SkiLifts', Key={'Lift': {'S': 'Lift 23'}, 'Metadata': {'S': 'Static Data'}}, AttributesToGet=['Lift', 'Metadata', 'Item'])

{'Item': {'Metadata': {'S': 'Static Data'}, 'Lift': {'S': 'Lift 23'}},
 'ResponseMetadata': {'RequestId': 'CCBT3FR88VFPRT7GVF8JUMECJRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 28 May 2023 22:36:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '64',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'CCBT3FR88VFPRT7GVF8JUMECJRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3339958042'},
  'RetryAttempts': 0}}